In [14]:
print(deepweeds_path)

/Users/hlyd4326/Documents/sih_projects/Weed-ID-Interchange/conversion_tools


In [22]:
"""
deepweeds_to_json.py

Convert a DeepWeeds .csv file to CCT format.

Modified from code from Microsoft's CameraTraps repo:
https://github.com/microsoft/CameraTraps
"""


"""Constants and environment"""

import pandas as pd
import uuid
import json
import time
import datetime
from tqdm import tqdm
import humanfriendly
import exifread
import sys
import os
import PIL
#from visualization import visualize_db
#import path_utils

deepweeds_path = str(os.path.realpath('..'))

input_metadata_file = deepweeds_path + "/deepweeds_to_json/labels.csv"
output_file = deepweeds_path + "/deepweeds_to_json/deepweeds_imageinfo.json"
image_base = deepweeds_path + "/deepweeds_to_json/deepweeds_images"

## Relative paths should be used; for now I am using absolute paths.
dirName = os.path.realpath('..')

#filename_replacements = {dirName:'DeepWeeds'}
category_mappings = {'none':'empty'}


"""
Read source data

DeepWeeds annotations have Filename, Label, and Species columns.
"""

input_metadata = pd.read_csv(input_metadata_file)

print('Read {} columns and {} rows from metadata file'.format(len(input_metadata.columns),
      len(input_metadata)))


"""Main loop over labels"""

startTime = time.time()

relativePathToImage = {}

images = []
annotations = []
categoryIDToCategories = {}
missingFiles = []

duplicateImageIDs = set()

# Force the empty category to be ID 0
emptyCat = {}
emptyCat['name'] = 'empty'
emptyCat['id'] = 9
categoryIDToCategories[0] = emptyCat

# iRow = 0; row = input_metadata.iloc[iRow]
for iRow,row in tqdm(input_metadata.iterrows(),total=len(input_metadata)):
    
    # ImageID,Filename,FilePath,SpeciesID
    imageID = str(row['Filename'])
    fn = row['Filename']
    relativePath = os.path.join(dirName,fn)
    
    # This makes an assumption of one annotation per image, which happens to be
    # true in this data set.
    if relativePath in relativePathToImage:

        im = relativePathToImage[relativePath]
        assert im['id'] == imageID
        duplicateImageIDs.add(imageID)
            
    else:
        im = {}
        im['id'] = imageID
        im['file_name'] = relativePath
        im['license'] = 2
        images.append(im)
        relativePathToImage[relativePath] = im
        
        fullPath = os.path.join(image_base,relativePath)
        
        if not os.path.isfile(fullPath):
            
            missingFiles.append(fullPath)
        
        else:
            # Retrieve image width and height
            pilImage = PIL.Image.open(fullPath)
            width, height = pilImage.size
            im['width'] = width
            im['height'] = height

    categoryName = row['Species'].lower()
    if categoryName in category_mappings:
        categoryName = category_mappings[categoryName]
        
    categoryID = row['Label']
    assert isinstance(categoryID,int)
    
    if categoryID not in categoryIDToCategories:
        category = {}
        category['name'] = row['Species'].lower()
        category['id'] = row['Label']
        categoryIDToCategories[categoryID] = category
   # else:
    #    assert categoryIDToCategories[categoryID]['name'] == categoryName
    
    # Create an annotation
    ann = {}
    
    ann['id'] = str(uuid.uuid1())
    ann['image_id'] = im['id']    
    ann['category_id'] = categoryID
    ann['agdata_id'] = 'deepweeds'
    
    annotations.append(ann)
    
categories = list(categoryIDToCategories.values())

elapsed = time.time() - startTime
print('Finished verifying file loop in {}, {} images, {} missing images, {} repeat labels'.format(
        humanfriendly.format_timespan(elapsed), len(images), len(missingFiles), len(duplicateImageIDs)))    


"""
Check for images that aren't included in the metadata file

This function is disabled for now.
"""

# Enumerate all images
# list(relativePathToImage.keys())[0]

#imageFullPaths = path_utils.find_images(image_base,bRecursive=True)
#unmatchedFiles = []

#for iImage,imagePath in enumerate(imageFullPaths):
    
#    fn = os.path.relpath(imagePath,image_base)    
#    if fn not in relativePathToImage:
#        unmatchedFiles.append(fn)

#print('Finished checking {} images to make sure they\'re in the metadata, found {} mismatches'.format(
#        len(imageFullPaths),len(unmatchedFiles)))

"""Create info object"""

info = {}
info['year'] = 2019
info['version'] = 1
info['description'] = 'COCO style database'
info['secondary_contributor'] = 'Converted to COCO .json by Henry Lydecker'
info['contributor'] = 'Alex Olsen'

"""Create license object"""
# Individual licence
license = {}
license['id'] = 2
license['name'] = 'apache2'
license['url'] = 'https://www.apache.org/licenses/LICENSE-2.0'
# Licenses object
licenses = {}
licenses['license'] = license


"""
Create dataset object

This information is invariant across a dataset upon upload.

Datasets can then be concatenated to include multiple agdatasets
"""

agdataset = {}
agdataset['agdata_id'] = 'deepweeds'
agdataset['crop_type'] = 'pastoral_grassland'
agdataset['upload_time'] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
agdataset['camera_type'] = 'weedlogger'
agdataset['camera_angle'] = '90'
agdataset['camera_fov'] = '85'
agdataset['emr_channels'] = "NA"

agdata = {}
agdata['agdataset'] = agdataset

"""Write output"""

json_data = {}
json_data['images'] = images
json_data['annotations'] = annotations
json_data['categories'] = categories
json_data['info'] = info
json_data['licenses'] = licenses
json_data['agdata'] = agdata
json.dump(json_data, open(output_file,'w'), indent=4)

print('Finished writing .json file with {} images, {} annotations, and {} categories'.format(
        len(images),len(annotations),len(categories)))

  8%|▊         | 1395/17509 [00:00<00:02, 6931.12it/s]

Read 3 columns and 17509 rows from metadata file


100%|██████████| 17509/17509 [00:02<00:00, 7107.13it/s]


Finished verifying file loop in 2.47 seconds, 17509 images, 17509 missing images, 0 repeat labels
Finished writing .json file with 17509 images, 17509 annotations, and 9 categories


## Utility Functions

For now I have moved these utility functions out of the convertor. They were difficult to get functioning and we may want to use different solutions once we are farther along.

In [3]:
"""Sanity-check the database's integrity"""

from data_management.databases import sanity_check_json_db

options = sanity_check_json_db.SanityCheckOptions()
sortedCategories,data = sanity_check_json_db.sanity_check_json_db(output_file, options)

    
"""Render a bunch of images to make sure the labels got carried along correctly"""
bbox_db_path = output_file
output_dir = preview_base

options = visualize_bbox_db.BboxDbVizOptions()
options.num_to_visualize = 1000
options.sort_by_filename = False

htmlOutputFile = visualize_bbox_db.process_images(bbox_db_path,output_dir,image_base,options)

ModuleNotFoundError: No module named 'data_management'